In [ ]:
import pandas as pd
import unicodedata

In [ ]:
df = pd.read_csv('statsmpg.csv', sep=';', encoding='latin1')
# Import the CSV file. After encountering an error with the default UTF-8 encoding,
# using Latin-1 resolves the issue and displays the data.

In [ ]:
df.head()

In [ ]:
#Header =2 to optimise the index

In [ ]:
# change the index of the header
df = pd.read_csv('statsmpg.csv', sep=';', encoding='latin1', header=2)
df.head()

In [ ]:
pd.set_option('display.max_columns', None)

df.describe()

# There are 124 columns. We need to check their type and delete the useless ones

In [ ]:
# Calculate the NaN rate for each column in percentage
taux_nan_par_colonne = df.isna().mean() * 100

# Display by group of 20
for i in range(0, len(taux_nan_par_colonne), 20):  # Divide by group of 20 columns
    print(taux_nan_par_colonne[i:i+20])
    print("\n")  # Add an empty line to separate the group

In [ ]:
print(df.dtypes)
#Visualize the type of data for each column

In [ ]:
#  Read the CSV file in batches of 1000 rows
for i in range(0, len(df.columns), 20):  # Divide by group of 20 columns
    print(df.dtypes[i:i+20])
    print("\n")  # Add an empty line to separate

In [ ]:
for i in range(0, len(df.columns), 20):print(df.isnull().sum()[i:i+20])  # display the number of nulle value
print("\n") 

In [ ]:
print(len(df))

In [ ]:
# 1) Normalise the columns name
def normalize_list(lst):
    return [unicodedata.normalize('NFC', c.strip()) for c in lst]

df.columns = normalize_list(df.columns)
#create the column that we will keep
colonnes_utiles = [
    "Joueur", "Poste", "Cote", "Var cote", "Enchère moy", "% achat", "% achat tour 1",
    "Nb match", "But", "%Titu", "Temps", "Tps moy", "Min/But", "Prix/but",
    "But/Peno", "Pass decis.", "Occas° créée", "Tirs", "Tirs cadrés", "Corner gagné",
    "Ballons", "Interceptions", "Tacles", "%Duel", "Fautes", "Dégagements",
    "Ballon perdu", "Grosse occas manquée", "But/Coup-franc", "Note"
]

# useful_columns = ["Player", "Position", "Rating", "Rating Change", 
# "Average Bid", "% Purchased", "% Purchased Round 1","Matches Played", "Goals", 
#"% Started", "Minutes Played", "Avg Minutes", "Min per Goal", "Price per Goal","Goals per Penalty",
# "Assists", "Chances Created", "Shots", "Shots on Target", "Corners Won","Touches", "Interceptions", "Tackles", 
#"% Duels Won", "Fouls", "Clearances","Balls Lost", "goals on freekick", "Big Missed Chances", "rate"

df = df[colonnes_utiles]

In [ ]:
df

In [ ]:
df_clean = df.fillna(0)
#replace the missing values by 0

In [ ]:
df_clean

In [ ]:
df_clean.isna().sum()
#check there is no missing values anymore


In [ ]:
for i in range(0, len(df.columns), 30):  
    print(df_clean.dtypes[i:i+30])
    print("\n")  

In [ ]:
cols_to_convert = ["Enchère moy", "% achat", "% achat tour 1", "%Titu", "Prix/but", "%Duel"]
df_clean[cols_to_convert] = df_clean[cols_to_convert].apply(pd.to_numeric, errors='coerce')
#Some columsn needs to be converted because their type doesn't match

In [ ]:
for i in range(0, len(df.columns), 30):  
    print(df_clean.dtypes[i:i+30])
    print("\n")  

In [ ]:
print(df_clean.isna().sum())

In [ ]:
df_clean['Enchère moy'] = df_clean['Enchère moy'].fillna(0)
df_clean['% achat'] = df_clean['% achat'].fillna(0)  # Replace with 0 if percentages are not informative
df_clean['% achat tour 1'] = df_clean['% achat tour 1'].fillna(0)  # Replace by 0
df_clean['%Titu'] = df_clean['%Titu'].fillna(0)  # Replace with 0 or a default value
df_clean['Prix/but'] = df_clean['Prix/but'].fillna(df_clean['Prix/but'].mean())
df_clean['%Duel'] = df_clean['%Duel'].fillna(0)  # Replace by 0


In [ ]:
print(df_clean.isna().sum())

In [ ]:
#Graph

In [ ]:
#Creating the value‑for‑money and performance columns.

# Create the Overall Performance column.
df_clean['Performance globale'] = df_clean['But'] + df_clean['Pass decis.'] + df_clean['Interceptions'] + df_clean['%Duel']

# Display the first rows to verify
print(df_clean[['Joueur', 'But', 'Pass decis.', 'Interceptions', '%Duel', 'Performance globale']].head())


# Create the value‑for‑money ratio column
df_clean['Rapport qualité/prix'] = df_clean['Performance globale'] / df_clean['Enchère moy']

# Display the first rows to verify
print(df_clean[['Joueur', 'Performance globale', 'Enchère moy', 'Rapport qualité/prix']].head())




In [ ]:
#Scatter plot

import seaborn as sns
import matplotlib.pyplot as plt

# Create the overall performance metric
df_clean['Performance globale'] = df_clean['But'] + df_clean['Pass decis.'] + df_clean['Interceptions'] + df_clean['%Duel']

# Scatterplot
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_clean, x='Enchère moy', y='Performance globale', hue='Poste', palette='viridis', s=100)
plt.title('Price vs global performance', fontsize=16)
plt.xlabel('Average price (average bid)', fontsize=12)
plt.ylabel('Global performance', fontsize=12)
plt.legend(title='position')
plt.grid(True)
plt.show()


In [ ]:
#2. Boxplot of prices by position

plt.figure(figsize=(10, 6))
sns.boxplot(data=df_clean, x='Poste', y='Enchère moy', hue='Poste', palette='Set2', legend=False)
plt.title('Repartition of price by position', fontsize=16)
plt.xlabel('Position', fontsize=12)
plt.ylabel('Average price (average bid)', fontsize=12)
plt.grid(True)
plt.show()


In [ ]:
#3 Histogram of the value‑for‑money ratio

# Create a value‑for‑money ratio metric
df_clean['Rapport qualité/prix'] = df_clean['Performance globale'] / df_clean['Enchère moy']

# Histogram
plt.figure(figsize=(10, 6))
sns.histplot(data=df_clean, x='Rapport qualité/prix', bins=30, kde=True, color='blue')
plt.title('Distribution of the value‑for‑money ratio', fontsize=16)
plt.xlabel('Value‑for‑money ratio', fontsize=12)
plt.ylabel('Number of players', fontsize=12)
plt.grid(True)
plt.show()

In [ ]:
#4. Barplot of the top 5 undervalued players by position
import numpy as np

# Replace infinite values with NaN
df_clean['Rapport qualité/prix'] = df_clean['Rapport qualité/prix'].replace([np.inf, -np.inf], np.nan)

# Remplace NaN by 0
df_clean['Rapport qualité/prix'] = df_clean['Rapport qualité/prix'].fillna(0)

# Sort the DataFrame by Position and value‑for‑money ratio (descending)
df_sorted = df_clean.sort_values(by=['Poste', 'Rapport qualité/prix'], ascending=[True, False])

# Select the top 5 players for each position
top_5_par_poste = df_sorted.groupby('Poste').head(5).reset_index(drop=True)

# Barplot
plt.figure(figsize=(12, 8))
sns.barplot(data=top_5_par_poste, x='Joueur', y='Rapport qualité/prix', hue='Poste', palette='coolwarm', dodge=False)
plt.title('Top 5 undervalued players by position', fontsize=16)
plt.xlabel('Player', fontsize=12)
plt.ylabel('Value‑for‑money ratio', fontsize=12)
plt.xticks(rotation=45)
plt.legend(title='Position')
plt.grid(True)

# Adjust the y‑axis limits if necessary
plt.ylim(0, top_5_par_poste['Rapport qualité/prix'].max() * 1.1)  # Add a 10% margin to the y‑axis

plt.show()

In [ ]:
#5 Correlation of key actions vs. price

# Select relevant columns
actions_cles = ['But', 'Pass decis.', 'Interceptions', '%Duel', 'Tirs cadrés', 'Cote'] # goal, assist, interceptions, %dual, shoots on target
df_heatmap = df_clean[actions_cles + ['Enchère moy']]


# Compute the correlation
correlation = df_heatmap.corr()

# matrix
plt.figure(figsize=(10, 6))
sns.heatmap(correlation, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation between key actions and average price', fontsize=16)
plt.show()

In [ ]:
#Descriptive statistics by position

# Descriptive statistics by position
stats_par_poste = df_clean.groupby('Poste')['Rapport qualité/prix'].agg(['mean', 'median', 'std', 'min', 'max'])
print(stats_par_poste)

In [ ]:
#Correlations between variables

# Compute the correlation matrix
correlation_matrix = df_clean[['Enchère moy', 'Performance globale', 'Temps', 'But', 'Pass decis.', 'Interceptions', '%Duel', 'Cote']].corr()

# average bid, global performance, time (played), goal, assist, interceptions, %dual

# Display the correlation heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation matrix', fontsize=16)
plt.show()

In [ ]:
#Performance by position

# Boxplot of overall performance by position
plt.figure(figsize=(10, 6))
sns.boxplot(data=df_clean, x='Poste', y='Performance globale', palette='Set2')
plt.title('Distribution of overall performance by position', fontsize=16)
plt.xlabel('Position', fontsize=12)
plt.ylabel('Global performance', fontsize=12)
plt.grid(True)
plt.show()

In [ ]:
#Most undervalued players

from scipy.stats import zscore

# Compute the z‑score of the value‑for‑money ratio
df_clean['Z-score'] = zscore(df_clean['Rapport qualité/prix'])

# Select the most undervalued players (negative z‑score)
joueurs_sous_cotes = df_clean[df_clean['Z-score'] < 0].sort_values(by='Z-score').head(10)
print(joueurs_sous_cotes[['Joueur', 'Poste', 'Rapport qualité/prix', 'Z-score']])

In [ ]:
#Performance and playtime

# Scatter plot: Playtime vs Overall Performance
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_clean, x='Temps', y='Performance globale', hue='Poste', palette='viridis', s=100)
plt.title('Playtime vs Overall Performance', fontsize=16)
plt.xlabel('Playtime', fontsize=12)
plt.ylabel('Globale performance', fontsize=12)
plt.legend(title='Position',
    loc='upper left',
    bbox_to_anchor=(1.02, 1),
    borderaxespad=0)
plt.grid(True)
plt.show()

In [ ]:
#Goals and assists

# Create a goals + assists metric
df_clean['Buts + Passes'] = df_clean['But'] + df_clean['Pass decis.']

# Scatter plot: Goals + Assists vs Price
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_clean, x='Buts + Passes', y='Enchère moy', hue='Poste', palette='viridis', s=100)
plt.title('Goals + Assists vs Price', fontsize=16)
plt.xlabel('Goals + Assists', fontsize=12)
plt.ylabel('Average price( average bid)', fontsize=12)
plt.legend(title='Position')
plt.grid(True)
plt.show()

In [ ]:
#Defenders and goalkeepers

# Filter central defenders (DC), fullbacks (DL), and goalkeepers (G
df_defenseurs_gardiens = df_clean[df_clean['Poste'].isin(['DC', 'DL', 'G'])]

# Ensure the ‘Position’ column exists and contains valid values
if 'Poste' in df_defenseurs_gardiens.columns:
    # Scatter plot: Interceptions vs Price
    plt.figure(figsize=(10, 6))
    sns.scatterplot(
        data=df_defenseurs_gardiens,
        x='Interceptions',
        y='Enchère moy',
        hue='Poste',  # Use ‘Position’ for color
        palette='Set2',
        s=100
    )
    plt.title('Interceptions vs Prix (DC, DL, G)', fontsize=16)
    plt.xlabel('Interceptions', fontsize=12)
    plt.ylabel('Prix moyen (Enchère moy)', fontsize=12)
    plt.legend(title='Poste')  # Add a legend
    plt.grid(True)
    plt.show()

In [ ]:
#Analysis based on bidding

# Scatter plot: Rating vs value‑for‑money ratio
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_clean, x='Cote', y='Rapport qualité/prix', hue='Poste', palette='coolwarm', s=100)
plt.title('Value vs value for money ratio', fontsize=16)
plt.xlabel('Value', fontsize=12)
plt.ylabel('Rapport qualité/prix', fontsize=12)
plt.legend(title='Position')
plt.grid(True)
plt.show()

In [ ]:
#Save the final file that will be used for data visualization with Power BI.

df_clean.to_csv('df_clean.csv', index=False)

In [ ]:
# Dataframe creation by position by removing variables empty or none for each dataframe

dataframes = {
    'Striker'          : df[df['Poste'] == 'A'],
    'Offensive mid'   : df[df['Poste'] == 'MO'],
    'Defensive mid'   : df[df['Poste'] == 'MD'],
    'Fullback' : df[df['Poste'] == 'DL'],
    'Centerback' : df[df['Poste'] == 'DC'],
    'Goalkeeper'            : df[df['Poste'] == 'G']
}

for name, dfposte in dataframes.items():
    dataframes[name] = dfposte.dropna(axis=1, how='all')

In [ ]:
# % of the missnig value

missing_dict = {}

# Loop for each dataframe
for name, df in dataframes.items():
    # % of missing values
    missing_values = df.isna().mean() * 100
    missing_values = missing_values[missing_values > 0]

    if not missing_values.empty:  # check
        missing_dict[name] = missing_values.round(2)

# Dataframe creation
final_result = pd.DataFrame(missing_dict).fillna(' ')

print('Pourcentage de valeurs manquantes')
display(final_result)

In [ ]:
with pd.ExcelWriter('stats_joueurs.xlsx', engine='openpyxl') as writer:
    for name, df_sheet in dataframes.items():
        df_sheet.to_excel(writer, sheet_name=name, index=True)
print("Le fichier 'stats_joueurs.xlsx' a été créé.")

In [ ]:
# categoraisation of the relevant stats to the posiiton

categories = {
    'Striker': [
        'But', 'Pass decis.', 'Occas° créée', 'Tirs', 'Tirs cadrés', 
        'Corner gagné', 'Grosse occas manquée', 'Balle non rattrapée', 'But/Peno', 'But/Coup-franc', 'Cote'
    ],
    'Offensive mid': [
        'But', 'Pass decis.', 'Occas° créée', 'Tirs', 'Tirs cadrés', 'Corner gagné', '%Passes',
        'Ballons', 'Fautes', 'Poss Mil', 'Centres', 'Dépossédé','Cote'
    ],
    'Defensive mid': [
        'Pass decis.', 'Interceptions', 'Tacles', '%Duel', 'Fautes', 'Poss Def', 'Ballons', 'Dépossédé', 'Cote'
    ],
    'Fullback': [
        'Cleansheet', 'Interceptions', 'Tacles', 'Centres', 'Centres ratés', 'Fautes', 'Poss Mil', 'Corner gagné', 'Cote'
    ],
    'Centerback': [
        'Cleansheet', 'Interceptions', 'Tacles', '%Duel', 'Fautes', 'Poss Def', 'But concédé', 'Erreur>But', 'Diff de buts', 'Cote'
    ],
    'Goalkeeper': [
        'Cleansheet', 'But évité', 'Action stoppée', 'Dégagements', 'But concédé', 'Plonge&stop', 'Erreur>But', 'Diff de buts', 'Cote'
    ]
}

columns_initiales = ['Joueur', 'Poste', 'Cote', 'Enchère moy', '% achat', 'Nb match']

dataframes_categorises = {}

fichier_excel = 'stats_poste.xlsx'

with pd.ExcelWriter(fichier_excel, engine='openpyxl') as writer:
    for poste, df_poste in dataframes.items():
        df_combined = df_poste[columns_initiales]

        cols_to_add = [col for col in categories[poste] if col in df_poste.columns]
        df_category = df_poste[cols_to_add]
        df_category = df_category.fillna(0)
        
        # concatenation of the common column 
        df_final = pd.concat([df_combined, df_category], axis=1)

   
        dataframes_categorises[poste] = df_final  

        df_final.to_excel(writer, sheet_name=poste, index=False)

print(f"the file'{fichier_excel}' has been created.")
print("The permanent Dataframes are stored in 'dataframes_categorises'.")

In [ ]:

nb_graphs = len(dataframes_categorises)
fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(30, 20)) 

fig.suptitle("Correlation matrix", fontsize=30)

axes = axes.flatten()

# Loop on Dataframes and display of heatmap
for i, (poste, df_final) in enumerate(dataframes_categorises.items()):
    if i >= 6:  # 6 graph limit
        break
    df_numeric = df_final.select_dtypes(include='number')
    corr_abs = df_numeric.corr().abs()

    sns.heatmap(corr_abs, annot=True, cmap='coolwarm', fmt='.1f', cbar=False, square=True, ax=axes[i])
    axes[i].set_title(f'{poste}', fontsize=24)  
    axes[i].tick_params(axis='x', rotation=60)

# space between graphs
plt.subplots_adjust(hspace=0, wspace=-.4, top=1) 

plt.tight_layout() 
plt.show();

In [ ]:
# Example of calculation of the variable score with the variable 'But' du DataFrame 'Attaquants'

from sklearn.preprocessing import MinMaxScaler

df_attaquants = dataframes_categorises['Striker']
scaler = MinMaxScaler(feature_range=(0, 100))
        
# Normalise only the column 'But'
df_attaquants['Score'] = scaler.fit_transform(df_attaquants[['But']])
        
# Round to the nearest integer
df_attaquants['Score'] = df_attaquants['Score'].round().astype(int)

# Strikers ranking by 'Score' descending
df_attaquants = df_attaquants.sort_values(by='Score', ascending=False)

# Select the columns 'Joueur', 'But' et 'Score'
df_affichage = df_attaquants[['Joueur', 'But', 'Score']]

print("\n The best 10 strikers following their score based on goals:")
display(df_affichage.head(10).style.hide(axis="index"))

In [ ]:
# Example of calculation of the variable score with the variable 'But' et 'Tirs du DataFrame 'Attaquants'

df_attaquants = dataframes_categorises['Striker']
scaler = MinMaxScaler(feature_range=(0, 100))

# Calculate individual scores
df_attaquants['score_But'] = scaler.fit_transform(df_attaquants[['But']])
df_attaquants['score_Tirs'] = scaler.fit_transform(df_attaquants[['Tirs']])

# Calculate globale score as average of both
df_attaquants['Score'] = ((df_attaquants['score_But'] + df_attaquants['score_Tirs']) / 2).round().astype(int)

# round the individual scores
df_attaquants['score_But'] = df_attaquants['score_But'].round().astype(int)
df_attaquants['score_Tirs'] = df_attaquants['score_Tirs'].round().astype(int)

# rank the strikers by descending 'Score'
df_attaquants = df_attaquants.sort_values(by='Score', ascending=False)


df_affichage = df_attaquants[['Joueur', 'But', 'Tirs', 'score_But', 'score_Tirs', 'Score']]

print("\nThe best 10 strikers following their score based on the goals and shoots:")
display(df_affichage.head(10).style.hide(axis="index"))

In [ ]:
from sklearn.preprocessing import MinMaxScaler

fichier_excel_score = 'stats_poste_score.xlsx'

with pd.ExcelWriter(fichier_excel_score, engine='openpyxl') as writer:
    for poste, df in dataframes_categorises.items():
        cols_score = [col for col in categories[poste] if col in df.columns]

        if cols_score:
            # 1) Conversion in float, the rest is NaN
            df_numeric = df[cols_score].apply(pd.to_numeric, errors='coerce')

            # 2) Ascending ranking  (1 = best)
            df_scores = df_numeric.rank(ascending=True,
                                       method='min',
                                       na_option='bottom')

            # 3) Inversion for negative metrics
            for col in cols_score:
                if col in inverse_metrics:
                    df_scores[col] = df_numeric[col].rank(
                        ascending=False,
                        method='min',
                        na_option='bottom'
                    )

            # 4) scale (0–100) for the average on rank
            scaler = MinMaxScaler(feature_range=(0, 100))
            moyennes = df_scores.mean(axis=1).values.reshape(-1, 1)
            df['Score'] = scaler.fit_transform(moyennes).round().astype(int)

            # 5) Descending triage
            df = df.sort_values(by='Score', ascending=False)

        df.to_excel(writer, sheet_name=poste, index=False)

        
        print(f"\nThe 5 {poste} by their score")
        display(df.head())

print(f"the file '{fichier_excel_score}' has been created.")

In [ ]:
# To prepare the filtered tables, a last filter needs to be realized

fichier_excel_filtré = 'stats_poste_filtré.xlsx'

# Function to get 90% of a variable maximum
def get_90_percent_of_max(df, col):
    return df[col].quantile(.9)

with pd.ExcelWriter(fichier_excel_filtré, engine='openpyxl') as writer:
    for poste, df in dataframes_categorises.items():
        # select the columns
        df_poste = df[columns_initiales + ['Score'] + categories[poste]]
        
        # Filter to keep the top 30 players depending on the score
        df_poste_sorted = df_poste.sort_values(by='Score', ascending=False)
        df_top_30 = df_poste_sorted.head(30)

        # Filter the players having a value or average bid less than 90% of the maximum value, % of buying less than 90, and rating > 4,5
        df_filtered = df_top_30[
            ((df_top_30['Cote'] < get_90_percent_of_max(df_poste, 'Cote')) | 
            (df_top_30['Enchère moy'] < get_90_percent_of_max(df_poste, 'Enchère moy'))) & 
            (df_top_30['% achat'] < 90) & 
            (df_top_30['Note'] > 4.5) &
            (df_top_30['Forme'] > 4.5)

]

        df_filtered.to_excel(writer, sheet_name=poste, index=False)

      
        print(f"\n Top 5 best{poste} after filtering")
        display(df_filtered.head())

print(f"the file '{fichier_excel_filtré}' has been created with the filtered players.")